In [1]:
import numpy as np
import pandas as pd
import math
import tensorflow as tf
import nltk
import re
import sklearn
from string import punctuation

from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.corpus import wordnet

!pip install contractions
import contractions

from sklearn.model_selection import train_test_split

from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.layers.experimental.preprocessing import TextVectorization

from sklearn.preprocessing import LabelEncoder
from sklearn.utils import shuffle

nltk.download('vader_lexicon')
from nltk.sentiment.vader import SentimentIntensityAnalyzer

!pip install num2words
from num2words import num2words

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 289.9/289.9 KB 6.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 103.2/103.2 KB 12.3 MB/s eta 0:00:00


[nltk_data] Downloading package vader_lexicon to /root/nltk_data...


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 125.2/125.2 KB 3.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for docopt: filename=docopt-0.6.2-py2.py3-none-any.whl size=13721 sha256=840e31624613d17175420e053ef3fdfa5ba047accaa205c479bd3564a0e06126
  Stored in directory: /root/.cache/pip/wheels/70/4a/46/1309fc853b8d395e60bafaf1b6df7845bdd82c95fd59dd8d2b
Successfully built docopt


In [2]:
nltk.download('wordnet')
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')
nltk.download('omw-1.4')

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...


True

In [3]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [4]:
train_file_path = '/content/gdrive/My Drive/Colab Notebooks/cs4248/Project/fulltrain.csv'
test_file_path = '/content/gdrive/My Drive/Colab Notebooks/cs4248/Project/balancedtest.csv'

In [5]:
train_df = pd.read_csv(train_file_path, names=['label', 'text'])
test_df = pd.read_csv(test_file_path, names=['label', 'text'])

In [6]:
train_df.head()

,label,text
0,1,"A little less than a decade ago, hockey fans w..."
1,1,The writers of the HBO series The Sopranos too...
2,1,Despite claims from the TV news outlet to offe...
3,1,After receiving 'subpar' service and experienc...
4,1,After watching his beloved Seattle Mariners pr...


## Preprocessing

In [7]:
# Preprocessing
stopwords = nltk.corpus.stopwords.words('english')
lemmatizer = WordNetLemmatizer()

def get_pos_tag_simplified(word):
  tag = nltk.pos_tag([word])[0][1][0]

  if tag == 'V':
    return wordnet.VERB
  elif tag == 'J':
    return wordnet.ADJ
  elif tag == 'R':
    return wordnet.ADV
  else:
    return wordnet.NOUN

def preprocessing(sentence):
  sentence = sentence.lower().strip().split()
  sentence = ' '.join([contractions.fix(word) for word in sentence])
  sentence = nltk.word_tokenize(sentence)
  sentence = ' '.join([lemmatizer.lemmatize(word, get_pos_tag_simplified(word)) for word in sentence if word not in punctuation and word not in stopwords])
  # sentence = ' '.join([word for word in sentence if word not in punctuation and word not in stopwords])
  
  temp = ''
  for word in sentence.split():
    if word.isnumeric():
      n2w = num2words(word).split()
      t = ''
      for w in n2w:
        if w != n2w[-1]:
          t += w + '-'
        else:
          t += w
      temp += ' ' + t
    else:
      temp += ' ' + word

  return temp
# train_df['text'] = train_df['text'].apply(preprocessing)

In [8]:
train_df.head()

,label,text
0,1,"A little less than a decade ago, hockey fans w..."
1,1,The writers of the HBO series The Sopranos too...
2,1,Despite claims from the TV news outlet to offe...
3,1,After receiving 'subpar' service and experienc...
4,1,After watching his beloved Seattle Mariners pr...


## Sentiment analysis

In [9]:
senti_analyzier = SentimentIntensityAnalyzer()

def get_senti(sentence):
  score = senti_analyzier.polarity_scores(sentence)['compound']

  if score > 0.05:
    return 1
  if score < -0.05:
    return -1
  return 0

In [10]:
train_df['senti'] = train_df['text'].apply(lambda x: get_senti(x))

## F1 score function

In [11]:
#F1 score function
def recall_s(y_true, y_pred):
    true_positives = tf.keras.backend.sum(tf.keras.backend.round(tf.keras.backend.clip(y_true * y_pred, 0, 1)))
    possible_positives = tf.keras.backend.sum(tf.keras.backend.round(tf.keras.backend.clip(y_true, 0, 1)))
    recall = true_positives / (possible_positives + tf.keras.backend.epsilon())
    return recall

def precision_s(y_true, y_pred):
    true_positives = tf.keras.backend.sum(tf.keras.backend.round(tf.keras.backend.clip(y_true * y_pred, 0, 1)))
    predicted_positives = tf.keras.backend.sum(tf.keras.backend.round(tf.keras.backend.clip(y_pred, 0, 1)))
    precision = true_positives / (predicted_positives + tf.keras.backend.epsilon())
    return precision

def f1(y_true, y_pred):
    precision = precision_s(y_true, y_pred)
    recall = recall_s(y_true, y_pred)
    return 2 * ((precision * recall) / (precision + recall + tf.keras.backend.epsilon()))

## Format y_train values

In [12]:
encoder =  LabelEncoder()
y1 = encoder.fit_transform(train_df['label'])
print(y1)

[0 0 0 ... 3 3 3]


In [13]:
Y = pd.get_dummies(y1).values
print(Y[0:5])

[[1 0 0 0]
 [1 0 0 0]
 [1 0 0 0]
 [1 0 0 0]
 [1 0 0 0]]


In [14]:
TOTAL_WORDS = 15000
MAX_LEN = 256
EMBEDDING_SIZE = 512

In [15]:
vectorize_layer = TextVectorization(
    max_tokens=TOTAL_WORDS,
    output_mode='int',
    output_sequence_length=MAX_LEN,
    ngrams=3
)
vectorize_layer.adapt(train_df['text'])

In [16]:
x_train, x_val, y_train, y_val = train_test_split(train_df, Y, test_size=0.2, shuffle=True)

### Model

In [17]:
text_input = tf.keras.Input(shape=(1,), dtype=tf.string, name='text')
text_layer = vectorize_layer(text_input)
text_layer = tf.keras.layers.Embedding(input_dim=TOTAL_WORDS, output_dim=EMBEDDING_SIZE)(text_layer)
text_layer = tf.keras.layers.GlobalAveragePooling1D(keepdims=True)(text_layer)
text_layer = tf.keras.layers.SimpleRNN(64, activation='tanh',
                              return_sequences=True,
                              dropout=0.5,
                              recurrent_dropout=0.5,
                              recurrent_regularizer=tf.keras.regularizers.L1(0.0001))(text_layer)
text_layer = tf.keras.layers.SimpleRNN(128, activation='tanh',
                              return_sequences=False,
                              dropout=0.5,
                              recurrent_dropout=0.5,
                              recurrent_regularizer=tf.keras.regularizers.L1(0.0001))(text_layer)
text_layer = tf.keras.layers.Dense(64, activation='relu')(text_layer)
text_layer = tf.keras.layers.Dropout(0.5)(text_layer)
text_layer = tf.keras.layers.Dense(16, activation='relu')(text_layer)

In [18]:
senti_normalizer = tf.keras.layers.Normalization(axis=None)
senti_normalizer.adapt(x_train['senti'])
senti_normalizer(train_df['senti'])

senti_input = tf.keras.Input(shape=(1,), dtype='int64', name='senti')
senti_layer = tf.keras.layers.Dense(16, activation='relu')(senti_input)

In [19]:
input = tf.keras.layers.concatenate([text_layer,
                                     senti_layer,
                                     ])

output = tf.keras.layers.Dense(4, activation='softmax', name='output')(input)
model = tf.keras.Model(inputs=[
    text_input,
    senti_input,
], outputs=[output])

model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['categorical_accuracy'])
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 text (InputLayer)              [(None, 1)]          0           []                               
                                                                                                  
 text_vectorization (TextVector  (None, 256)         0           ['text[0][0]']                   
 ization)                                                                                         
                                                                                                  
 embedding (Embedding)          (None, 256, 512)     7680000     ['text_vectorization[0][0]']     
                                                                                                  
 global_average_pooling1d (Glob  (None, 1, 512)      0           ['embedding[0][0]']          

In [20]:
callback = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=3)

history = model.fit({'text': x_train['text'],
                     'senti': x_train['senti'],
                    },
                    {'output': y_train},
                    epochs=50,
                    batch_size=256,
                    validation_data=([x_val['text'],
                                      x_val['senti'],
                                      ],
                                      y_val),
                    callbacks = [callback],
                    verbose=1)

Epoch 1/50
153/153 [==============================] - 51s 269ms/step - loss: 0.9225 - categorical_accuracy: 0.6468 - val_loss: 0.3930 - val_categorical_accuracy: 0.8628
Epoch 2/50
153/153 [==============================] - 31s 199ms/step - loss: 0.3009 - categorical_accuracy: 0.8986 - val_loss: 0.1958 - val_categorical_accuracy: 0.9337
Epoch 3/50
153/153 [==============================] - 26s 171ms/step - loss: 0.1451 - categorical_accuracy: 0.9555 - val_loss: 0.1565 - val_categorical_accuracy: 0.9501
Epoch 4/50
153/153 [==============================] - 24s 154ms/step - loss: 0.0915 - categorical_accuracy: 0.9723 - val_loss: 0.1612 - val_categorical_accuracy: 0.9531
Epoch 5/50
153/153 [==============================] - 23s 153ms/step - loss: 0.0729 - categorical_accuracy: 0.9783 - val_loss: 0.1755 - val_categorical_accuracy: 0.9512
Epoch 6/50
153/153 [==============================] - 20s 134ms/step - loss: 0.0571 - categorical_accuracy: 0.9838 - val_loss: 0.1978 - val_categorical_acc

## Testing of model

In [21]:
y_test = encoder.fit_transform(test_df['label'])
y_test = pd.get_dummies(y_test).values

In [22]:
#test_df['text'] = test_df['text'].apply(preprocessing)
test_df['senti'] = test_df['text'].apply(lambda x: get_senti(x))

In [23]:
test_history = model.evaluate([test_df['text'],
                               test_df['senti'],
                               ], y_test, verbose=1, batch_size=128)

24/24 [==============================] - 1s 49ms/step - loss: 2.2067 - categorical_accuracy: 0.6547


In [24]:
!pip install shap
import shap

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 572.4/572.4 KB 9.6 MB/s eta 0:00:00


In [43]:
explainer = shap.DeepExplainer(model, [x_train['text'].values[:100], x_train['senti'].values[:100]])

Your TensorFlow version is newer than 2.4.0 and so graph support has been removed in eager mode and some static graphs may not be supported. See PR #1483 for discussion.


In [45]:
x_test = [test_df['text'].values[:10], test_df['senti'].values[:10]]

In [47]:
shap_values = explainer.shap_values(x_test)

`tf.keras.backend.set_learning_phase` is deprecated and will be removed after 2020-10-11. To update it, simply pass a True/False value to the `training` argument of the `__call__` method of your layer or model.


AttributeError: ignored

In [ ]:
# tf.keras.models.save_model(model, '/content/gdrive/My Drive/Colab Notebooks/cs4248/Project/rnn.h5')
model.save('/content/gdrive/My Drive/Colab Notebooks/cs4248/Project/rnn')

In [ ]:
new_model = tf.keras.models.load_model('/content/gdrive/My Drive/Colab Notebooks/cs4248/Project/rnn')

In [ ]:
new_model

## Saving of dataframe

In [ ]:
# train_df.to_csv(train_file_path, index=False)
# test_df.to_csv(test_file_path, index=False)